In [1]:
# Libraries
import pandas as pd
import random
import json
from datasets import load_dataset
import re
import math
import random
import spacy

In [2]:
dataset = load_dataset("gsm8k", "main")
dataset["test"][5]

Found cached dataset gsm8k (/Users/iv/.cache/huggingface/datasets/gsm8k/main/1.1.0/37bfb08b1d4fcbb01f06b03d9e1ef5f1fcbd4d3af3d08842c50d7305091285ba)


  0%|          | 0/2 [00:00<?, ?it/s]

{'question': 'Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?',
 'answer': 'The discount price of one glass is 60/100 * 5 = $<<60/100*5=3>>3.\nIf every second glass is cheaper, that means Kylar is going to buy 16 / 2 = <<16/2=8>>8 cheaper glasses.\nSo for the cheaper glasses, Kylar is going to pay 8 * 3 = $<<8*3=24>>24.\nAnd for the regular-priced glasses, Kylar will pay 8 * 5 = $<<8*5=40>>40.\nSo in total Kylar needs to pay 24 + 40 = $<<24+40=64>>64 for the glasses he wants to buy.\n#### 64'}

In [3]:
model_name = "TheBloke/Llama-2-7B-Chat-GGUF"
model_basename = "./llama-2-7b-chat.Q5_K_M.gguf"

In [4]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [5]:
model_path = hf_hub_download(repo_id=model_name, filename=model_basename)
llm = Llama(
            model_path=model_path,
            n_threads=2,
            n_batch=512,
            n_gpu_layers=32
            )

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/iv/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/./llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  4096,  4096,     1,

In [6]:
def get_random_test_object(dataset):
    if "test" in dataset and len(dataset["test"]) > 0:
        random_index = random.randint(0, len(dataset["test"]) - 1)
        test_object = dataset["test"][random_index]
        
        # Assuming each object in the dataset has 'question' and 'answer' keys
        question = test_object.get("question", "No question found")
        answer = test_object.get("answer", "No answer found")
        
        return question, answer
    else:
        return None, None

In [7]:
def get_random_mutation(csv_file_path):
    try:
        df = pd.read_csv(csv_file_path, header=None, encoding='utf-8', delimiter='.') 
    except UnicodeDecodeError:
        df = pd.read_csv(csv_file_path, header=None, encoding='ISO-8859-1', delimiter='.') 

    random_prompt = random.choice(df[1].tolist())
    return random_prompt


In [8]:
def get_random_mutation_txt(txt_file_path):
    with open(txt_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove any leading/trailing whitespace and filter out empty lines
    prompts = [line.strip() for line in lines if line.strip()]
    
    if prompts:
        return random.choice(prompts)
    else:
        return "No mutation prompts found."

In [9]:
# write the task description here:
task_description = "Generate an instruction on how to solve the problem, based on the given question "

In [10]:
task_description1 = "Generate an instruction, or advice on how to solve a problem"

#### Hide

In [11]:
# getting mutated prompts
question, answer = get_random_test_object(dataset)

In [12]:
print(question)

Jackie is trying to decide whether to do her taxes herself or hire an accountant. If she does the taxes herself, she'll be able to do 3 fewer hours of freelance work, losing $35/hour in missed income. The accountant charges $90. How much more money will she have if she hires the accountant?


In [13]:
print(answer)

First find the total lost revenue if Jackie does her taxes herself: $35/hour * 3 hours = $<<35*3=105>>105
Then subtract the accountant's charge to find how much money Janet saves: $105 - $90 = $<<105-90=15>>15
#### 15


## Instruction Generation

In [11]:
def check_answer(answer, output):
    if answer == output:
        return 1
    return 0

In [12]:
def get_logic(answer):
    return re.findall("<<(.*?)>>", answer)

In [13]:
def tokenise_logic(logic):
    tokenised_logic = []
    operators = "+-/*="
    current_token = ""
    
    for c in logic:
        if c in operators:
            tokenised_logic.append(current_token)
            tokenised_logic.append(c)
            current_token = ""
        elif c == '.' or c.isnumeric():
            current_token += c
    tokenised_logic.append(current_token)
    
    return tokenised_logic

In [14]:
def check_logic_sentence(line, logic_tokens):
    i = 0

    for token in logic_tokens:
        if token in line:
            i += 1

    if len(logic_tokens) == i:
        return True
    return False

In [19]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

class llamathwiz:
    def __init__(self):
        # self.tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
        # self.model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
        self.gen = 0
        self.llm = llm
        
    def generate_instructions(self, question, task_description, max_tokens=100):
        formatted_input = f"{task_description} {question}"
        output = self.llm(formatted_input)

        return output["choices"][0]["text"]
    
    def generate_instructions1(self, num_instructions, max_tokens=1000, temperature=1.5):
        instructions = []
        for num in range(num_instructions):
            formatted_input = "Generate an instruction on how to solve a simple math problem without the need of a question"
            generated_text = self.llm(formatted_input)["choices"][0]["text"]
            instructions.append(generated_text)

        return instructions

#         formatted_input = f"Generate {num_instructions} separate instructions on how to solve a simple math problem without the need of a question."
#         generated_text = self.llm(formatted_input, max_tokens=2048)

#         return generated_text

    def process_with_llm(self, prompt):
        return self.llm(prompt, max_tokens=1000)
    
    def apply_mutation(self, instruction, mutation_prompt):
        # Example mutation - this can be customized based on your mutation logic
        return self.llm(f"{instruction} \n Rewrite this and apply the following mutation. Mutation: {mutation_prompt}")["choices"][0]["text"]
    
    def fitness(self, database_answer, output):
        final_answer = database_answer.split()[-1]

        score = 0

        logic = get_logic(database_answer)

        tokenised_logic_sentences = []

        for l in logic:
            tokenised_logic_sentences.append(tokenise_logic(l))

        total_score = 1 + len(tokenised_logic_sentences)

        for line in output.split("."):
            for sentence in tokenised_logic_sentences:
                if check_logic_sentence(line, sentence):
                    tokenised_logic_sentences.pop(tokenised_logic_sentences.index(sentence))
                    score += 1
                    break
        last_line = line

        num_answer = 'a'

        for word in last_line.split(" "):
            try:
                num_answer = str(int(word))
            except:
                pass

        score += check_answer(final_answer, num_answer)
        return score/total_score
    
    def iteration_first(self, num_instructions=10):
        self.gen = 1
        self.generated_prompts = []
        self.generated_answers = []
        self.iteration_questions = []
        self.scores = []
        
        mutation_prompt = get_random_mutation_txt("./Prompt-Engineering-OpenDI/mutation_prompts.txt")
        for _ in range(num_instructions):
            question, database_answer = get_random_test_object(dataset)  # Fetch a random question from your dataset
            self.iteration_questions.append((question, database_answer))
            instruction = self.generate_instructions1(1)
            # mutated_instruction = self.apply_mutation(instruction, mutation_prompt)
            # self.generated_prompts.append(mutated_instruction)
            self.generated_prompts.append(instruction)
            processed_output = self.process_with_llm(f'Q:{question} I:{instruction} A:') 
            self.generated_answers.append(processed_output)
            self.scores.append(self.fitness(database_answer, processed_output))
            
    
    def iteration_execute(self):
        self.gen += 1
        self.generated_prompts = []
        self.generated_answers = []
        self.iteration_questions = []
        self.scores = []

        mutation_prompt = get_random_mutation_txt("./Prompt-Engineering-OpenDI/mutation_prompts.txt")
        for instruction in self.best_instructions:
            question, database_answer = get_random_test_object(dataset)  # Fetch a random question from your dataset
            self.iteration_questions.append((question, database_answer))
            mutated_instruction = self.apply_mutation(instruction, mutation_prompt)
            self.generated_prompts.append(mutated_instruction)
            processed_output = self.process_with_llm(f'Q:{question} I:{mutated_instruction} A:')
            self.generated_answers.append(processed_output)
            self.scores.append(self.fitness(database_answer, processed_output))
    
    def iteration_prepare(self):
        self.best_instructions = []
        self.find_best_scores(self.scores)
        self.best_instructions.append(self.replace_pos(self.best_instructions[0], self.best_instructions[1], ['ADV', 'ADJ', 'NOUN']))
        self.best_instructions.append(self.replace_pos(self.best_instructions[1], self.best_instructions[0], ['ADV', 'ADJ', 'NOUN']))
        for instr in self.generate_instructions1(4):
            self.best_instructions.append(instr)
        for i in range(2):
            mutation_prompt = get_random_mutation_txt("./Prompt-Engineering-OpenDI/mutation_prompts.txt")
            self.best_instructions.append(self.apply_mutation(self.best_instructions[i], mutation_prompt))
        
    def find_best_scores(self, scores):
        for i in range(2):
            maximum_val = 0
            maximum_index = 0
            while i < len(scores):
                if scores[i] > maximum_val:
                    maximum_val = scores[i]
                    maximum_index = i
                i += 1
            scores.pop(maximum_index)
            print(self.generated_prompts[maximum_index])
            self.best_instructions.append(self.generated_prompts.pop(maximum_index))
            
    def replace_pos(self, string1, string2, POS):

        """
        Replace tokens of specific parts of speech (POS) in string1 with corresponding
        tokens of the same POS from string2.

        Parameters:
        - string1 (str): The input string where certain POS will be replaced.
        - string2 (str): The reference string from which POS replacements will be taken.
        - POS (list): A list of POS tags to identify which tokens to replace in string1.

        Returns:
        - str: The modified string with replacements of tokens based on specified POS tags.

        Possible POS:
        "ADJ": "adjective",
        "ADP": "adposition",
        "ADV": "adverb",
        "AUX": "auxiliary",
        "CONJ": "conjunction",
        "CCONJ": "coordinating conjunction",
        "DET": "determiner",
        "INTJ": "interjection",
        "NOUN": "noun",
        "NUM": "numeral",
        "PART": "particle",
        "PRON": "pronoun",
        "PROPN": "proper noun",
        "PUNCT": "punctuation",
        "SCONJ": "subordinating conjunction",
        "SYM": "symbol",
        "VERB": "verb".
        """
        nlp = spacy.load("en_core_web_sm")
        doc1 = nlp(string1)
        doc2 = nlp(string2)

        new_tokens = []

        for token1 in doc1:
            if token1.pos_ in POS:
                #generating a list of all matching tokens
                matching_tokens = [token2.text for token2 in doc2 if token2.pos_ == token1.pos_]
                #use a random token if possible, or else use the same
                if matching_tokens:
                    new_token = random.choice(matching_tokens)
                else:
                    new_token = token1.text
            else:
                new_token = token1.text
            new_tokens.append(new_token)

        # Join the modified tokens to form the final string
        result = ' '.join(new_tokens)
        return result
    
    

In [20]:
ai = llamathwiz()

In [21]:
output = ai.process_with_llm("What is 2+2?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    9289.24 ms
llama_print_timings:      sample time =       0.30 ms /     1 runs   (    0.30 ms per token,  3289.47 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    5516.85 ms /     1 runs   ( 5516.85 ms per token,     0.18 tokens per second)
llama_print_timings:       total time =    5530.76 ms


In [22]:
print(output)

{'id': 'cmpl-3877d944-63df-4e0d-a83f-208bb2bf1d96', 'object': 'text_completion', 'created': 1703190454, 'model': '/Users/iv/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/./llama-2-7b-chat.Q5_K_M.gguf', 'choices': [{'text': '', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 8, 'completion_tokens': 0, 'total_tokens': 8}}


In [89]:
ai.iteration_first()

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3603.53 ms
llama_print_timings:      sample time =      17.52 ms /    40 runs   (    0.44 ms per token,  2283.24 tokens per second)
llama_print_timings: prompt eval time =    3683.24 ms /    18 tokens (  204.62 ms per token,     4.89 tokens per second)
llama_print_timings:        eval time =    3762.06 ms /    39 runs   (   96.46 ms per token,    10.37 tokens per second)
llama_print_timings:       total time =    7718.83 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3603.53 ms
llama_print_timings:      sample time =      72.07 ms /   128 runs   (    0.56 ms per token,  1776.05 tokens per second)
llama_print_timings: prompt eval time =    1695.81 ms /   133 tokens (   12.75 ms per token,    78.43 tokens per second)
llama_print_timings:        eval time =   12345.88 ms /   127 runs   (   97.21 ms per token,    10.29 tokens per second)
llama_print_timings:       total time =   15

AttributeError: 'dict' object has no attribute 'split'

In [88]:
# self.generated_prompts = []
# self.generated_answers = []
# self.iteration_questions = []
# self.scores = []

ai.scores

[]

In [202]:
ai.generated_answers[1]

'Mandy owes Benedict $100, and they agreed to have a monthly interest of 2%. To find out how much Mandy should give to Benedict after 3 months, we can use the formula: Principal + Interest = Total Amount Due. In this case, the principal is $100, and the interest is 2% of the principal x time = 2% x 3 months = $6. So, the total amount due is $100 + $6 = $106. Therefore, Mandy should give Benedict $106 after 3'

In [203]:
ai.iteration_questions[1][1]

'The monthly interest is $100 x 2/100 = $<<100*2/100=2>>2.\nAfter 3 months, the total interest amount to $2 x 3 = $<<2*3=6>>6.\nTherefore, Mandy should give Benedict a total of $100 + $6 = $<<100+6=106>>106.\n#### 106'

In [65]:
ai.iteration_prepare()

[' mark.\nFor example: To solve for x in the equation 2x + 5 = 11, follow these steps:\n\nStep 1: Subtract 5 from both sides of the equation.\nStep 2: Divide both sides of the equation by 2.\nStep 3: The solution is x = 3.\n\nInstructions for solving the simple math problem:\nTo solve for x in the equation 4x - 7 = 19, follow these steps:\n\nStep 1: Add 7 to both sides of the equation.\nStep']
[' or equation.\n\nExample: To solve the problem 2+2, follow these steps:\n1. Take the number 2 and add it to itself (i.e., 2 + 2 =).\n2. The result is 4.\n\nSo, the solution to the problem 2+2 is 4.\n\nPlease let me know if you have any questions or need further clarification.']


Llama.generate: prefix-match hit

llama_print_timings:        load time =    3603.53 ms
llama_print_timings:      sample time =      81.38 ms /   128 runs   (    0.64 ms per token,  1572.81 tokens per second)
llama_print_timings: prompt eval time =    2087.90 ms /    18 tokens (  115.99 ms per token,     8.62 tokens per second)
llama_print_timings:        eval time =   11719.91 ms /   127 runs   (   92.28 ms per token,    10.84 tokens per second)
llama_print_timings:       total time =   14851.83 ms
Llama.generate: prefix-match hit

Llama.generate: prefix-match hit
llama_print_timings:        load time =    3603.53 ms
llama_print_timings:      sample time =      79.23 ms /   128 runs   (    0.62 ms per token,  1615.59 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   12110.82 ms /   128 runs   (   94.62 ms per token,    10.57 tokens per second)
llama_print

KeyboardInterrupt: 

In [55]:
ai.best_instructions[0]

' or equation.\n\nFor example: "To solve this problem, follow these steps: 1) identify the variables involved, 2) plug them into the equation, 3) simplify and solve for the answer."\n\nPlease let me know if you have any questions or need further clarification.'

In [135]:
ai.iteration_execute()

Llama.generate: prefix-match hit

llama_print_timings:        load time =    8529.41 ms
llama_print_timings:      sample time =      19.04 ms /    31 runs   (    0.61 ms per token,  1628.24 tokens per second)
llama_print_timings: prompt eval time =    5046.00 ms /   358 tokens (   14.09 ms per token,    70.95 tokens per second)
llama_print_timings:        eval time =    3271.80 ms /    30 runs   (  109.06 ms per token,     9.17 tokens per second)
llama_print_timings:       total time =    8687.36 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =    8529.41 ms
llama_print_timings:      sample time =      96.94 ms /   128 runs   (    0.76 ms per token,  1320.45 tokens per second)
llama_print_timings: prompt eval time =    3396.64 ms /   233 tokens (   14.58 ms per token,    68.60 tokens per second)
llama_print_timings:        eval time =   14213.54 ms /   127 runs   (  111.92 ms per token,     8.94 tokens per second)
llama_print_timings:       total time =   19

In [141]:
ai.generated_prompts[4]

'Encourage Creativity: For creative writing prompts like ’Write a story about X,’ an improved version could be, ’Write a fantasy story about X set in a world where Y is possible.’  mark.\nFor example: To find 3x, multiply 3 by x.\n\nSolution:\nTo find 2x, multiply 2 by x.'

In [40]:
def average(lst): 
    return sum(lst) / len(lst)

In [41]:
average(ai.scores)

0.0

In [66]:
scores_avg = []
gens = []

ai.iteration_first()
for i in range(10):
    scores_avg.append(average(ai.scores))
    gens.append(ai.gen)
    ai.iteration_prepare()
    ai.iteration_execute()
    
ai.score()

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3603.53 ms
llama_print_timings:      sample time =      79.56 ms /   128 runs   (    0.62 ms per token,  1608.77 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   13212.48 ms /   128 runs   (  103.22 ms per token,     9.69 tokens per second)
llama_print_timings:       total time =   14266.16 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3603.53 ms
llama_print_timings:      sample time =      12.53 ms /    16 runs   (    0.78 ms per token,  1276.73 tokens per second)
llama_print_timings: prompt eval time =    2637.38 ms /   221 tokens (   11.93 ms per token,    83.80 tokens per second)
llama_print_timings:        eval time =    1412.12 ms /    15 runs   (   94.14 ms per token,    10.62 tokens per second)
llama_print_timings:       total time =    4

KeyboardInterrupt: 

In [49]:
gens

[]

In [56]:
instructions = ai.generate_instructions1(3)

TypeError: Llama.__call__() got an unexpected keyword argument 'num_return_sequences'

In [18]:
print(instruction)


Solution: To determine how many times Peter can go to the movies, we need to divide his total budget by the cost of one movie ticket and popcorn. Since the cost of one movie ticket is $7 and the cost of one bag of popcorn is also $7, we can calculate the number of movies he can afford as follows:
42 dollars / 7 dollars = 6 times
Therefore, Peter can go to the movies 6 times with the budget he has for the week.


In [25]:
for instr in instructions:
    print(instr + '\n')

 step by step.

Solution:
Step 1: Read the Problem Carefully
The first step in solving any math problem is to read it carefully. Make sure you understand what is being asked and what information is given. Pay attention to any symbols, units, or keywords that are used in the problem.

Step 2: Identify the Type of Problem
Once you have read the problem, try to identify the type of problem you are dealing with. Is it a multiplication problem, division problem, addition problem, subtraction problem, etc.? This will help you determine the best strategy for solving the


Please provide me with an example of a math problem that you would like me to generate instructions for, and I will do my best to create step-by-step instructions on how to solve it.
 step by step:

Step 1: Read the Problem Carefully
The first step in solving any math problem is to read it carefully. Make sure you understand what is being asked and what information you need to provide in your solution. Pay attention to any g

In [29]:
print(instructions)

 using the concept of "similarity"

Answer: Sure! Here are three instructions on how to solve a math problem using the concept of similarity:

Instruction 1: Use Similar Shapes to Find the Missing Side Length

Problem: In the figure below, two similar triangles have been drawn. The length of one side of each triangle is given, as well as the ratio of their corresponding sides. Find the missing side length of the second triangle using the concept of similarity.

Solution: To solve this problem, we can use the fact that similar triangles have proportional side lengths. Since


In [49]:
instructions["choices"][0]["text"]

"\nStep-by-step guide for solving a simple math problem:\n\nStep 1: Identify the problem type\nThe first step in solving a math problem is to identify the type of problem you are dealing with. Is it an addition, subtraction, multiplication, or division problem? Make sure you understand the operation required to solve the problem.\n\nStep 2: Gather information and simplify the problem\nOnce you have identified the type of problem, gather all the necessary information from the problem statement. Simplify the problem by removing any unnecessary factors or variables. This will make it easier to solve the problem.\n\nStep 3: Solve the problem using the appropriate operation\nNow that you have gathered all the necessary information and simplified the problem, it's time to solve it! Use the appropriate operation (addition, subtraction, multiplication, or division) to find the solution. Be sure to perform the operation correctly and accurately.\n\nConclusion: Check your answer and simplify it 

In [186]:
def fitness(database_answer, output):
    final_answer = database_answer.split()[-1]

    score = 0

    logic = get_logic(database_answer)

    tokenised_logic_sentences = []

    for l in logic:
        tokenised_logic_sentences.append(tokenise_logic(l))
    print(tokenised_logic_sentences)

    total_score = 1 + len(tokenised_logic_sentences)

    for line in output.split("."):
        for sentence in tokenised_logic_sentences:
            if check_logic_sentence(line, sentence):
                tokenised_logic_sentences.pop(tokenised_logic_sentences.index(sentence))
                score += 1
                break
    last_line = line
    print(tokenised_logic_sentences)

    num_answer = 'a'

    for word in last_line.split(" "):
        try:
            num_answer = str(int(word))
        except:
            pass

    score += check_answer(final_answer, num_answer)
    return score/total_score

In [187]:
fitness('The $100 was divided into 3 + 2 = <<3+2=5>>5 parts.\nSo each part is $100/5 = $<<100/5=20>>20.\nSo Gerald received $20 x 3 = $<<20*3=60>>60.\nGerald bought a book at $<<10=10>>10.\nTherefore, Gerald was left with $60 - $10 = $<<60-10=50>>50.\n#### 50',
"2 + 3 = 5. Easy peasy, right? Now let's see how many apps Travis has on his tablet now:\nA:Travis has 61 - 9 = 52 apps on his tablet after deleting 9 apps he didn't use anymore. Then, he downloaded 18 more apps, so he has 52 + 18 = 70 apps on his tablet now!")

[['3', '+', '2', '=', '5'], ['100', '/', '5', '=', '20'], ['20', '*', '3', '=', '60'], ['10', '=', '10'], ['60', '-', '10', '=', '50']]
[['100', '/', '5', '=', '20'], ['20', '*', '3', '=', '60'], ['10', '=', '10'], ['60', '-', '10', '=', '50']]


0.16666666666666666